# 4.0 Backtrader Backtest

Run and analyze the short strategy using backtrader for more realistic simulation.

In [1]:
import sys
sys.path.insert(0, '.')

import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

## 1. Run Backtest

In [ ]:
from trading.bt_runner import run_backtest
from trading.config import DEFAULT_CONFIG

# Configuration
MODEL_PATH = Path('data/model_final.pt')
DATA_DIR = Path('data')

# Date range - TEST SET (model was trained on data before 2024-10-22)
# Train: 2021-01-13 to 2024-05-01
# Val:   2024-05-02 to 2024-10-21
# Test:  2024-10-22 to 2025-12-18
START_DATE = '2024-10-22'
END_DATE = '2025-12-18'

# Options
INITIAL_CASH = 100_000
MAX_SYMBOLS = None  # None = all symbols (slow), set to 500-1000 for faster testing
USE_DD_SCALING = True
USE_CONFIDENCE = False

In [3]:
%%time

result = run_backtest(
    model_path=MODEL_PATH,
    data_dir=DATA_DIR,
    start_date=START_DATE,
    end_date=END_DATE,
    initial_cash=INITIAL_CASH,
    use_dd_scaling=USE_DD_SCALING,
    use_confidence=USE_CONFIDENCE,
    max_symbols=MAX_SYMBOLS,
)

INFO - Starting backtest
INFO -   Model: data/model_final.pt
INFO -   Data: data
INFO -   Period: 2024-10-22 to 2025-12-18
INFO - Loaded model from data/model_final.pt
INFO -   Price features: 9
INFO -   Fund features: 19
INFO -   Embedding dims: 768
INFO - Loading price data from data/prices.pqt...
INFO -   Loaded 5,888,410 rows
INFO -   Splitting into per-symbol DataFrames...
INFO -     Processed 1,000/5,644 symbols...
INFO -     Processed 2,000/5,644 symbols...
INFO -     Processed 3,000/5,644 symbols...
INFO -     Processed 4,000/5,644 symbols...
INFO -     Processed 5,000/5,644 symbols...
INFO -   Done: 5,644 symbols loaded
INFO - Adding data feeds to cerebro...
INFO -   Processed 1,000/5,644 symbols (843 added)...
INFO -   Processed 2,000/5,644 symbols (1,690 added)...
INFO -   Processed 3,000/5,644 symbols (2,531 added)...
INFO -   Processed 4,000/5,644 symbols (3,449 added)...
INFO -   Processed 5,000/5,644 symbols (4,417 added)...
INFO -   Done: 5,028 data feeds added (616 ski

CPU times: user 25min 21s, sys: 8.33 s, total: 25min 29s
Wall time: 25min 5s


## 2. Results Summary

In [4]:
print("=" * 50)
print("BACKTEST RESULTS")
print("=" * 50)
print(f"Period:       {START_DATE} to {END_DATE}")
print(f"Initial Cash: ${INITIAL_CASH:,.0f}")
print(f"Final Value:  ${result.final_value:,.2f}")
print(f"Total Return: {result.total_return*100:.1f}%")
print(f"Sharpe Ratio: {result.sharpe_ratio:.2f}" if result.sharpe_ratio else "Sharpe Ratio: N/A")
print(f"Max Drawdown: {result.max_drawdown*100:.1f}%")
print(f"Total Trades: {result.n_trades}")
print("=" * 50)

BACKTEST RESULTS
Period:       2024-10-22 to 2025-12-18
Initial Cash: $100,000
Final Value:  $28,295.08
Total Return: -71.7%
Sharpe Ratio: -1.00
Max Drawdown: 78.2%
Total Trades: 57


## 3. Detailed Analyzer Output

In [5]:
# Trade analysis
trades = result.analyzers.get('trades', {})

if trades:
    print("Trade Analysis:")
    print(f"  Total trades: {trades.get('total', {}).get('total', 0)}")
    print(f"  Open trades:  {trades.get('total', {}).get('open', 0)}")
    print(f"  Closed trades: {trades.get('total', {}).get('closed', 0)}")
    
    won = trades.get('won', {})
    lost = trades.get('lost', {})
    
    if won.get('total', 0) + lost.get('total', 0) > 0:
        win_rate = won.get('total', 0) / (won.get('total', 0) + lost.get('total', 0))
        print(f"  Win rate: {win_rate*100:.1f}%")
        print(f"  Avg win:  ${won.get('pnl', {}).get('average', 0):.2f}")
        print(f"  Avg loss: ${lost.get('pnl', {}).get('average', 0):.2f}")

Trade Analysis:
  Total trades: 57
  Open trades:  5
  Closed trades: 52
  Win rate: 53.8%
  Avg win:  $894.65
  Avg loss: $-4035.79


In [6]:
# Drawdown analysis
dd = result.analyzers.get('drawdown', {})

if dd:
    print("Drawdown Analysis:")
    print(f"  Max drawdown: {dd.get('max', {}).get('drawdown', 0):.2f}%")
    print(f"  Max money down: ${dd.get('max', {}).get('moneydown', 0):,.2f}")
    print(f"  Max duration: {dd.get('max', {}).get('len', 0)} bars")

Drawdown Analysis:
  Max drawdown: 78.19%
  Max money down: $90,443.17
  Max duration: 19 bars


In [7]:
# Returns analysis
returns = result.analyzers.get('returns', {})

if returns:
    print("Returns Analysis:")
    print(f"  Total return: {returns.get('rtot', 0)*100:.2f}%")
    print(f"  Avg daily return: {returns.get('ravg', 0)*100:.4f}%")
    print(f"  Annualized return: {returns.get('rnorm100', 0):.2f}%")

Returns Analysis:
  Total return: -126.25%
  Avg daily return: -0.4280%
  Annualized return: -65.99%


## 4. Compare Configurations

Run multiple backtests with different settings.

In [8]:
# Define configurations to test
configs = [
    {"name": "Baseline (no scaling)", "use_dd_scaling": False, "use_confidence": False},
    {"name": "DD Scaling", "use_dd_scaling": True, "use_confidence": False},
    # {"name": "Confidence Weighted", "use_dd_scaling": False, "use_confidence": True},
    # {"name": "DD + Confidence", "use_dd_scaling": True, "use_confidence": True},
]

# Set to smaller number for comparison runs
COMPARE_MAX_SYMBOLS = MAX_SYMBOLS  # Use same as main backtest

In [ ]:
%%time

comparison_results = []

for cfg in configs:
    print(f"\nRunning: {cfg['name']}...")
    
    res = run_backtest(
        model_path=MODEL_PATH,
        data_dir=DATA_DIR,
        start_date=START_DATE,
        end_date=END_DATE,
        initial_cash=INITIAL_CASH,
        use_dd_scaling=cfg['use_dd_scaling'],
        use_confidence=cfg['use_confidence'],
        max_symbols=COMPARE_MAX_SYMBOLS,
    )
    
    comparison_results.append({
        'name': cfg['name'],
        'return': res.total_return,
        'sharpe': res.sharpe_ratio,
        'max_dd': res.max_drawdown,
        'n_trades': res.n_trades,
        'final_value': res.final_value,
    })

INFO - Starting backtest
INFO -   Model: data/model_final.pt
INFO -   Data: data
INFO -   Period: 2024-10-22 to 2025-12-18
INFO - Loaded model from data/model_final.pt
INFO -   Price features: 9
INFO -   Fund features: 19
INFO -   Embedding dims: 768
INFO - Loading price data from data/prices.pqt...



Running: Baseline (no scaling)...


INFO -   Loaded 5,888,410 rows
INFO -   Splitting into per-symbol DataFrames...
INFO -     Processed 1,000/5,644 symbols...


In [ ]:
# Display comparison
comp_df = pd.DataFrame(comparison_results)
comp_df['return_pct'] = comp_df['return'] * 100
comp_df['max_dd_pct'] = comp_df['max_dd'] * 100

print("\n" + "=" * 70)
print("CONFIGURATION COMPARISON")
print("=" * 70)
print(comp_df[['name', 'return_pct', 'sharpe', 'max_dd_pct', 'n_trades']].to_string(index=False))

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Returns
ax = axes[0]
ax.bar(comp_df['name'], comp_df['return_pct'])
ax.set_ylabel('Return (%)')
ax.set_title('Total Return')
ax.tick_params(axis='x', rotation=45)

# Sharpe
ax = axes[1]
sharpe_vals = comp_df['sharpe'].fillna(0)
ax.bar(comp_df['name'], sharpe_vals)
ax.set_ylabel('Sharpe Ratio')
ax.set_title('Sharpe Ratio')
ax.tick_params(axis='x', rotation=45)

# Max DD
ax = axes[2]
ax.bar(comp_df['name'], comp_df['max_dd_pct'])
ax.set_ylabel('Max Drawdown (%)')
ax.set_title('Max Drawdown')
ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 5. Compare with Notebook Backtest

Load results from the original notebook backtest for comparison.

In [ ]:
# Load notebook backtest results if available
notebook_results_path = DATA_DIR / 'short_backtest_improved.pqt'

if notebook_results_path.exists():
    nb_results = pd.read_parquet(notebook_results_path)
    print("Notebook backtest results:")
    display(nb_results)
else:
    print(f"No notebook results found at {notebook_results_path}")

## 6. Notes

**Key differences between backtrader and notebook backtests:**

1. **Order execution**: Backtrader simulates real order flow with fills on next bar
2. **Slippage**: Orders execute at open/close prices, not idealized prices
3. **Commission**: Applied per trade
4. **Short selling**: Proper margin and borrow mechanics

**Performance notes:**
- Loading all ~5000 symbols is slow (several minutes)
- Use `max_symbols` for faster iteration during development
- For accurate results, run with all symbols